In [24]:
import keras

In [51]:
import tensorflow as tf

In [48]:
from keras.optimizers import SGD


In [26]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.wrappers.scikit_learn import KerasClassifier
import keras.utils
import keras.layers
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier

In [27]:
import pandas as pd

In [28]:
weather = pd.read_csv("weather-daily.csv")

In [29]:
weather.head(5)

,Date,Cloud coverage,Visibility,Temperature,Dew point,Relative humidity,Wind speed,Station pressure,Altimeter,Solar energy,(Inverters)
0,01-02-16,0.103750,9.447917,3.113750,0.321667,79.454583,4.694583,29.228750,30.019583,20256,NaN
1,02-02-16,0.798333,3.941250,6.992500,6.215417,93.597500,13.290833,28.911667,29.697917,1761,NaN
2,03-02-16,0.864583,8.699583,1.615000,0.022917,85.001667,16.726250,29.026667,29.814583,2775,NaN
3,04-02-16,0.372917,10.000000,-2.472500,-5.886250,74.522500,9.455417,29.455000,30.254583,28695,NaN
4,05-02-16,0.515417,9.206667,-2.002917,-4.152083,82.027500,5.920000,29.550833,30.351250,9517,NaN


In [30]:
weather.shape

(637, 11)

In [31]:
df = pd.DataFrame(weather, columns = ['Date', 'Cloud coverage', 'Visibility', 'Temperature', 'Dew point',
       'Relative humidity', 'Wind speed', 'Station pressure', 'Altimeter',
       'Solar energy', '(Inverters)']) 

In [32]:
#seperating day, month and year
df['Day'] = pd.DatetimeIndex(df['Date']).day
df['Month'] = pd.DatetimeIndex(df['Date']).month
df['Year'] = pd.DatetimeIndex(df['Date']).year

In [33]:
df.shape

(637, 14)

In [34]:
df.columns

Index(['Date', 'Cloud coverage', 'Visibility', 'Temperature', 'Dew point',
       'Relative humidity', 'Wind speed', 'Station pressure', 'Altimeter',
       'Solar energy', '(Inverters)', 'Day', 'Month', 'Year'],
      dtype='object')

In [35]:
X=df.iloc[:, 0:15]

In [36]:
X.drop(["(Inverters)","Date"], axis = 1, inplace = True) 

In [37]:
X.shape

(637, 12)

In [38]:
columns = (['Cloud coverage', 'Visibility', 'Temperature', 'Dew point',
       'Relative humidity', 'Wind speed', 'Station pressure', 'Altimeter',
       'Day', 'Month', 'Year','Solar energy'])

In [39]:
Y = pd.DataFrame(X, columns=columns)

In [40]:
Y.shape

(637, 12)

In [41]:
# train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
#train_test_split(X, y, test_size=0.33, random_state=42)


In [42]:
model = keras.Sequential()

In [43]:
# For a mean squared error regression problem
#model.compile(optimizer='rmsprop',
#             loss='mse')
X_train.columns

Index(['Cloud coverage', 'Visibility', 'Temperature', 'Dew point',
       'Relative humidity', 'Wind speed', 'Station pressure', 'Altimeter',
       'Solar energy', 'Day', 'Month', 'Year'],
      dtype='object')

In [44]:
X_train.shape

(426, 12)

In [45]:
y_train.shape

(426, 12)

In [52]:
config = tf.ConfigProto(
    device_count={'GPU': 1},
    intra_op_parallelism_threads=1,
    allow_soft_placement=True
)

config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.6

session = tf.Session(config=config)

keras.backend.set_session(session)

In [55]:
from keras.optimizers import RMSprop,Adam


model.add(Dense(64, activation='relu', input_shape=(12,)))
model.add(Dense(8, activation='relu', input_shape=(16,)))
model.add(Dense(12, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])
history = model.fit(X_train, y_train,
                    epochs=150,batch_size=32,
                    verbose=1,
                    validation_data=(X_test, y_test))


Train on 426 samples, validate on 211 samples
Epoch 1/150
426/426 [==============================] - 2s 4ms/step - loss: 60021.2332 - acc: 0.5282 - val_loss: 56048.0291 - val_acc: 0.9810
Epoch 2/150
426/426 [==============================] - 0s 410us/step - loss: 58898.2929 - acc: 0.9648 - val_loss: 55491.0918 - val_acc: 0.9810
Epoch 3/150
426/426 [==============================] - 0s 383us/step - loss: 58328.6269 - acc: 0.9648 - val_loss: 54967.9532 - val_acc: 0.9810
Epoch 4/150
426/426 [==============================] - 0s 435us/step - loss: 57775.8310 - acc: 0.9648 - val_loss: 54446.9912 - val_acc: 0.9810
Epoch 5/150
426/426 [==============================] - 0s 352us/step - loss: 57222.1497 - acc: 0.9648 - val_loss: 53928.2761 - val_acc: 0.9810
Epoch 6/150
426/426 [==============================] - 0s 399us/step - loss: 56678.6878 - acc: 0.9648 - val_loss: 53420.6548 - val_acc: 0.9810
Epoch 7/150
426/426 [==============================] - 0s 422us/step - loss: 56136.2573 - acc: 0.9

426/426 [==============================] - 0s 460us/step - loss: 15560.9977 - acc: 0.9648 - val_loss: 14902.4791 - val_acc: 0.9810
Epoch 114/150
426/426 [==============================] - 0s 368us/step - loss: 15382.9414 - acc: 0.9648 - val_loss: 14738.9664 - val_acc: 0.9810
Epoch 115/150
426/426 [==============================] - 0s 296us/step - loss: 15209.0214 - acc: 0.9648 - val_loss: 14576.4203 - val_acc: 0.9810
Epoch 116/150
426/426 [==============================] - 0s 563us/step - loss: 15038.6936 - acc: 0.9648 - val_loss: 14417.5640 - val_acc: 0.9810
Epoch 117/150
426/426 [==============================] - 0s 516us/step - loss: 14870.1332 - acc: 0.9648 - val_loss: 14260.3627 - val_acc: 0.9810
Epoch 118/150
426/426 [==============================] - 0s 354us/step - loss: 14706.2033 - acc: 0.9648 - val_loss: 14108.8520 - val_acc: 0.9810
Epoch 119/150
426/426 [==============================] - 0s 435us/step - loss: 14545.7077 - acc: 0.9648 - val_loss: 13960.4649 - val_acc: 0.9810

In [56]:
# evaluate the model
scores = model.evaluate(X_test,y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 98.10%


In [57]:
from keras.models import load_model
model.save("solar_model.h5")
m = load_model("solar_model.h5")

In [62]:
y_pred = m.predict_classes(X,verbose=1)


637/637 [==============================] - 0s 77us/step


In [64]:
y_pred = model.predict_classes(X,verbose=1)
y_pred

637/637 [==============================] - 0s 687us/step


array([11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11

In [69]:
import sys
import os
import argparse


In [70]:
# Create the outputs folder - save any outputs you want managed by AzureML here
os.makedirs('./outputs', exist_ok=True)